In [96]:
import pandas as pd
import numpy as np

#Dados importados do perfil Vini0804 em 14/04/2024

diary_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\diary.csv")
likes_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\likes\films.csv")
ratings_df= pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\ratings.csv")
watched_df=pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\watched.csv")

# Manipulando os dados para juntar todos em um só dataframe 

watched_df =  watched_df.drop('Date', axis=1)  # Removendo a coluna Date pois ela não sera usada
watched_df["Like"]=watched_df["Name"].isin(likes_df["Name"]) # Criando a  coluna Like com base nos filmes que estao no dataframe Likes


#print(watched_df)

In [97]:
# Adicionando a data de quando o filme foi visto 

watched_df['First Log Date'] = np.nan  # Criando uma nova coluna 

for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_diario, filme_diario in enumerate(diary_df["Name"]):
        if filme == filme_diario:
            watched_df.loc[indice_watch, "First Log Date"] = diary_df.loc[indice_diario, "Watched Date"]
            
#print(watched_df)

In [98]:
# Adicionando o rating do filme 

watched_df['Rating'] = np.nan  # Criando uma nova coluna 

for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_rating, filme_rating in enumerate(ratings_df["Name"]):
        if filme == filme_rating:
            watched_df.loc[indice_watch, "Rating"] = ratings_df.loc[indice_rating, "Rating"]
            
#print(watched_df)

In [99]:
# Contador de filmes reassistidos

watched_df['Rewatch'] = 0
for indice_watch, filme in enumerate(watched_df["Name"]):
    for indice_diario, filme_diario in enumerate(diary_df["Name"]):
        if filme == filme_diario and diary_df["Rewatch"][indice_diario]=="Yes":
            watched_df.loc[indice_watch, 'Rewatch'] += 1

#print(watched_df)


In [100]:
 todos_os_filmes_df = pd.read_csv(r"C:\Users\arauj\OneDrive\Área de Trabalho\Movie_Project\TMDB_movie_dataset_v11.csv")

In [101]:
todos_os_filmes_df.columns


Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')

In [104]:
import pandas as pd
# Colunas interessantes: revenue, runtime, budget, original_language, overview, poster_path, popularity, genres, production_companies

# Colunas que usarei por hora: runtime, original_language, genres, production_companies
# Supondo que "Name" seja a coluna de filmes em watched_df e "title" seja a coluna correspondente em todos_os_filmes_df

#watched_df['Duração (min)'] = pd.merge(watched_df, todos_os_filmes_df, left_on='Name', right_on='title', how='left')['runtime'] # Errado


# Convertendo a coluna 'release_date' para formato de data

todos_os_filmes_df['release_date'] = pd.to_datetime(todos_os_filmes_df['release_date'])
todos_os_filmes_df['ano'] = todos_os_filmes_df['release_date'].dt.year
watched_df['ano'] = watched_df['ano'].astype(float)

watched_df = watched_df.rename(columns={'Year': 'ano', 'Name': 'title'})

filmes_filtrados = todos_os_filmes_df[todos_os_filmes_df['title'].isin(watched_df['title'])] # Filtra os filmes que estão no dataframe watched no dataframe todos_os_filmes

filmes_df = pd.merge(watched_df, filmes_filtrados, on=['title', 'ano'], how='inner')



            
#print(watched_df)

In [105]:
filmes_df


,title,ano,Letterboxd URI,Like,First Log Date,Rating,Rewatch,id,vote_average,vote_count,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,Captain Marvel,2019.0,https://boxd.it/9vSA,False,NaN,3.0,0,299537,6.843,14657,...,Captain Marvel,The story follows Carol Danvers as she becomes...,50.399,/AtsgWhDnHTq68L0lLsUrCnM7TjG.jpg,Higher. Further. Faster.,"Action, Adventure, Science Fiction",Marvel Studios,United States of America,English,"superhero, based on comic, alien invasion, sup..."
1,Spider-Man: Into the Spider-Verse,2018.0,https://boxd.it/azpY,True,NaN,5.0,0,324857,8.404,14024,...,Spider-Man: Into the Spider-Verse,Struggling to find his place in the world whil...,80.675,/iiZZdoQBEYBv6id8su7ImL0oCbD.jpg,More than one wears the mask.,"Action, Adventure, Animation, Science Fiction","Columbia Pictures, Lord Miller, Pascal Picture...",United States of America,"English, Japanese, Spanish","superhero, based on comic, aftercreditsstinger..."
2,The Favourite,2018.0,https://boxd.it/d2hI,False,NaN,4.0,0,375262,7.537,4855,...,The Favourite,"England, early 18th century. The close relatio...",18.954,/cwBq0onfmeilU5xgqNNjJAMPfpw.jpg,NaN,"Drama, Comedy, Thriller","Waypoint Entertainment, Element Pictures, Scar...","Ireland, United Kingdom, United States of America",English,"great britain, queen, rivalry, rabbit, lgbt, 1..."
3,Green Book,2018.0,https://boxd.it/hErk,False,NaN,3.5,0,490132,8.242,10542,...,Green Book,"Tony Lip, a bouncer in 1962, is hired to drive...",34.768,/7BsvSuDQuoqhWmU2fL7W2GOcZHU.jpg,Inspired by a True Friendship,"Drama, Comedy","Participant, DreamWorks Pictures, Cinetic Media",United States of America,"English, Italian, Russian","friendship, southern usa, road trip, racism, p..."
4,A Star Is Born,2018.0,https://boxd.it/aXgc,False,NaN,3.5,0,332562,7.503,11079,...,A Star Is Born,Seasoned musician Jackson Maine discovers — an...,27.891,/wrFpXMNBRj2PBiN4Z5kix51XaIZ.jpg,NaN,"Music, Drama, Romance","Thunder Road, 22 & Green, Gerber Pictures, Joi...",United States of America,English,"concert, country music, waitress, pop star, se..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1641,Two Irenes,2017.0,https://boxd.it/fuE0,False,2024-04-05,NaN,0,436449,6.900,38,...,As Duas Irenes,"In the shimmering heat of Brazil, 13-year-old ...",2.933,/iBJPgg8aiwtRjNLCwjwJLftGjV2.jpg,One name. One secret.,Drama,Lacuna Filmes,Brazil,Portuguese,father daughter relationship
1642,Before the Devil Knows You're Dead,2007.0,https://boxd.it/20Jy,True,2024-04-06,NaN,0,7972,7.099,1342,...,Before the Devil Knows You're Dead,When two brothers organize the robbery of thei...,20.301,/bEvwL1ZLG1tzoCdYM8qUGR6IG4o.jpg,No one was supposed to get hurt.,"Crime, Drama, Thriller","Michael Cerenzie Productions, Capitol Films, L...",United States of America,English,"daughter, career, sibling relationship, office..."
1643,Priest,2011.0,https://boxd.it/16fM,False,2024-04-09,NaN,0,38321,5.765,2070,...,Priest,"In an alternate world, humanity and vampires h...",19.594,/btNni3pWy3LTmckilBPxs3r8zDP.jpg,The War is Eternal. His Mission is Just the Be...,"Action, Fantasy, Horror, Thriller","Michael De Luca Productions, Stars Road Entert...",United States of America,"Spanish, English, Portuguese","vampire, dystopia, post-apocalyptic future, cr..."
1644,Philadelphia,1993.0,https://boxd.it/1XMk,False,2024-04-11,NaN,0,9800,7.725,3909,...,Philadelphia,Two competing lawyers join forces to sue a pre...,17.371,/tFe5Yoo5zT495okA49bq1vPPkiV.jpg,No one would take on his case... until one man...,Drama,TriStar Pictures,United States of America,English,"philadelphia, pennsylvania, court, jurors, aid..."
